In [1]:
import os
import glob
import torch
import random
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, Dataset


In [2]:
# Define relevant variables for the ML task
batch_size = 32
num_classes = 5
learning_rate = 0.001
num_epochs = 200
#  Device will determine whether to run the training on GPU or CPU.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
data_path = './'
train_file_name = 'train_dataset.csv'
test_file = 'val_dataset.csv'
train_csv_file = os.path.join(data_path,train_file_name)
val_csv_file = os.path.join(data_path,test_file)


In [4]:
# example
data = pd.read_csv(train_csv_file)
n=1
label1 = data.iloc[2,1]
file = data.iloc[n,0]
print('Label is:',label1)
# print('File is:',file)

Label is: 3


In [5]:
class CsvDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self,idx):
#         if torch.is_tensor(idx):
#             idx = idx.tolist()
        label= self.data.iloc[idx,1]
        np_file = self.data.iloc[idx,0]
        np_data = np.load(np_file)
        np_data = torch.from_numpy(np_data).float()
        if self.transform:
            sample = self.transform(np_data)
        return np_data,label

In [6]:
#######################################################
#                  Create Dataset
#######################################################

train_dataset = CsvDataset(train_csv_file)
train_loader = DataLoader(train_dataset, batch_size, shuffle=True)

valid_dataset = CsvDataset(val_csv_file)
valid_loader = DataLoader(valid_dataset,batch_size, shuffle=False)

dataiter = iter(train_loader)
files, labels = dataiter.next()
# print(files.shape)
# print(labels.shape)

In [7]:
class Net(nn.Module):
    def __init__(self, num_classes):
        super(Net, self).__init__()
        self.conv_layer1 = nn.Conv2d(in_channels=3, out_channels=18, kernel_size=(3,13))
        self.batch1 = nn.BatchNorm2d(18)
        self.conv_layer2 = nn.Conv2d(in_channels=18, out_channels=32, kernel_size=(3,13))
        self.batch2 = nn.BatchNorm2d(32)
        self.conv_layer3 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(3,13))
    
        self.sig = nn.Sigmoid()
        self.fc1 = nn.Linear(79872, 164)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(164, num_classes)
    
    def forward(self, x):
        out = self.conv_layer1(x)
        out = self.batch1(out)
        out = self.conv_layer2(out)
        out = self.batch2(out)
        out = self.conv_layer3(out)

        out = out.reshape(out.size(0), -1)
        out = self.sig(out)
#         out = self.relu1(out)
        out = self.fc1(out)
        out = self.sig(out)
#         out = self.relu1(out)
        out = self.fc2(out)
        return out

In [8]:
net = Net(num_classes)

if torch.cuda.is_available():
    net = net.cuda()
    
# print(net)

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
total_step = len(train_loader)

In [10]:
# net.train(True)

In [11]:
min_valid_loss = np.inf

for epoch in range(num_epochs):  # loop over the dataset multiple times
#     last_loss = 0.
    running_loss = 0.0
    for i, data in enumerate(train_loader):
        # get the inputs; data is a list of [inputs, labels]
        inputs,labels = data
        if torch.cuda.is_available():
            inputs = inputs.to(device)
            labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # print statistics
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))
        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0
        net.eval()     # Optional when not using Model Specific layer
        with torch.no_grad():
            for data, labels in valid_loader:
                if torch.cuda.is_available():
                    data, labels = data.cuda(), labels.cuda()

                target = net(data)
                loss = criterion(target,labels)
                valid_loss = loss.item() * data.size(0)

            print(f'Epoch {epoch+1} \t\t Training Loss: {running_loss / len(train_loader)} \t\t Validation Loss: {valid_loss / len(valid_loader)}')
            if min_valid_loss > valid_loss:
                print(f'Validation Loss Decreased({min_valid_loss:.6f}--->{valid_loss:.6f}) \t Saving The Model')
                min_valid_loss = valid_loss
                # Saving State Dict
                torch.save(net.state_dict(), 'saved_model.pth')
print('Finished Training')

Epoch [1/200], Loss: 1.6763
Epoch 1 		 Training Loss: 0.8381484746932983 		 Validation Loss: 17.126739621162415
Validation Loss Decreased(inf--->34.253479) 	 Saving The Model
Epoch [1/200], Loss: 1.6272
Epoch 1 		 Training Loss: 1.651739478111267 		 Validation Loss: 16.887189745903015
Validation Loss Decreased(34.253479--->33.774379) 	 Saving The Model
Epoch [2/200], Loss: 1.5726
Epoch 2 		 Training Loss: 0.7863002419471741 		 Validation Loss: 16.852104663848877
Validation Loss Decreased(33.774379--->33.704209) 	 Saving The Model
Epoch [2/200], Loss: 1.5439
Epoch 2 		 Training Loss: 1.5582659244537354 		 Validation Loss: 16.866197526454926
Epoch [3/200], Loss: 1.4516
Epoch 3 		 Training Loss: 0.7258102297782898 		 Validation Loss: 16.798275411128998
Validation Loss Decreased(33.704209--->33.596551) 	 Saving The Model
Epoch [3/200], Loss: 1.5275
Epoch 3 		 Training Loss: 1.4895465970039368 		 Validation Loss: 17.090402841567993
Epoch [4/200], Loss: 1.4360
Epoch 4 		 Training Loss: 0.717

Epoch [31/200], Loss: 1.2495
Epoch 31 		 Training Loss: 1.2852943539619446 		 Validation Loss: 15.416554033756256
Validation Loss Decreased(31.459991--->30.833108) 	 Saving The Model
Epoch [32/200], Loss: 1.2724
Epoch 32 		 Training Loss: 0.6362215876579285 		 Validation Loss: 14.835847735404968
Validation Loss Decreased(30.833108--->29.671695) 	 Saving The Model
Epoch [32/200], Loss: 1.2581
Epoch 32 		 Training Loss: 1.2652496099472046 		 Validation Loss: 14.642529308795929
Validation Loss Decreased(29.671695--->29.285059) 	 Saving The Model
Epoch [33/200], Loss: 1.2406
Epoch 33 		 Training Loss: 0.6202808618545532 		 Validation Loss: 14.581679284572601
Validation Loss Decreased(29.285059--->29.163359) 	 Saving The Model
Epoch [33/200], Loss: 1.2310
Epoch 33 		 Training Loss: 1.235766887664795 		 Validation Loss: 14.462006986141205
Validation Loss Decreased(29.163359--->28.924014) 	 Saving The Model
Epoch [34/200], Loss: 1.2426
Epoch 34 		 Training Loss: 0.6212916970252991 		 Validati

Epoch [54/200], Loss: 0.6841
Epoch 54 		 Training Loss: 0.6577446758747101 		 Validation Loss: 8.537067264318466
Validation Loss Decreased(17.363265--->17.074135) 	 Saving The Model
Epoch [55/200], Loss: 0.5696
Epoch 55 		 Training Loss: 0.2848229706287384 		 Validation Loss: 8.338926047086716
Validation Loss Decreased(17.074135--->16.677852) 	 Saving The Model
Epoch [55/200], Loss: 0.7042
Epoch 55 		 Training Loss: 0.6369201242923737 		 Validation Loss: 8.182781159877777
Validation Loss Decreased(16.677852--->16.365562) 	 Saving The Model
Epoch [56/200], Loss: 0.6109
Epoch 56 		 Training Loss: 0.3054312765598297 		 Validation Loss: 8.089734345674515
Validation Loss Decreased(16.365562--->16.179469) 	 Saving The Model
Epoch [56/200], Loss: 0.6138
Epoch 56 		 Training Loss: 0.6123262643814087 		 Validation Loss: 7.907097280025482
Validation Loss Decreased(16.179469--->15.814195) 	 Saving The Model
Epoch [57/200], Loss: 0.5956
Epoch 57 		 Training Loss: 0.29780495166778564 		 Validation 

Epoch [77/200], Loss: 0.3142
Epoch 77 		 Training Loss: 0.34277620911598206 		 Validation Loss: 4.1016926765441895
Validation Loss Decreased(8.266568--->8.203385) 	 Saving The Model
Epoch [78/200], Loss: 0.3340
Epoch 78 		 Training Loss: 0.16699141263961792 		 Validation Loss: 3.9855676889419556
Validation Loss Decreased(8.203385--->7.971135) 	 Saving The Model
Epoch [78/200], Loss: 0.3450
Epoch 78 		 Training Loss: 0.33948804438114166 		 Validation Loss: 3.955611437559128
Validation Loss Decreased(7.971135--->7.911223) 	 Saving The Model
Epoch [79/200], Loss: 0.2885
Epoch 79 		 Training Loss: 0.14425687491893768 		 Validation Loss: 3.9684009701013565
Epoch [79/200], Loss: 0.3785
Epoch 79 		 Training Loss: 0.33350059390068054 		 Validation Loss: 3.915887251496315
Validation Loss Decreased(7.911223--->7.831775) 	 Saving The Model
Epoch [80/200], Loss: 0.4105
Epoch 80 		 Training Loss: 0.20526956021785736 		 Validation Loss: 3.894114911556244
Validation Loss Decreased(7.831775--->7.78823

Epoch [101/200], Loss: 0.2676
Epoch 101 		 Training Loss: 0.1338152140378952 		 Validation Loss: 2.4827275201678276
Validation Loss Decreased(5.067043--->4.965455) 	 Saving The Model
Epoch [101/200], Loss: 0.2089
Epoch 101 		 Training Loss: 0.23826587200164795 		 Validation Loss: 2.4652264416217804
Validation Loss Decreased(4.965455--->4.930453) 	 Saving The Model
Epoch [102/200], Loss: 0.2779
Epoch 102 		 Training Loss: 0.138955757021904 		 Validation Loss: 2.4336883574724197
Validation Loss Decreased(4.930453--->4.867377) 	 Saving The Model
Epoch [102/200], Loss: 0.1909
Epoch 102 		 Training Loss: 0.23439448326826096 		 Validation Loss: 2.421597272157669
Validation Loss Decreased(4.867377--->4.843195) 	 Saving The Model
Epoch [103/200], Loss: 0.2370
Epoch 103 		 Training Loss: 0.11849869042634964 		 Validation Loss: 2.397298850119114
Validation Loss Decreased(4.843195--->4.794598) 	 Saving The Model
Epoch [103/200], Loss: 0.2285
Epoch 103 		 Training Loss: 0.23273873329162598 		 Vali

Epoch [124/200], Loss: 0.1881
Epoch 124 		 Training Loss: 0.18422143161296844 		 Validation Loss: 1.8382566198706627
Validation Loss Decreased(3.687237--->3.676513) 	 Saving The Model
Epoch [125/200], Loss: 0.1785
Epoch 125 		 Training Loss: 0.08924950659275055 		 Validation Loss: 1.8458170294761658
Epoch [125/200], Loss: 0.1866
Epoch 125 		 Training Loss: 0.1825457066297531 		 Validation Loss: 1.838682509958744
Epoch [126/200], Loss: 0.1884
Epoch 126 		 Training Loss: 0.09418752044439316 		 Validation Loss: 1.8497748970985413
Epoch [126/200], Loss: 0.1729
Epoch 126 		 Training Loss: 0.18062859028577805 		 Validation Loss: 1.8374722748994827
Validation Loss Decreased(3.676513--->3.674945) 	 Saving The Model
Epoch [127/200], Loss: 0.1973
Epoch 127 		 Training Loss: 0.09865885227918625 		 Validation Loss: 1.8113978505134583
Validation Loss Decreased(3.674945--->3.622796) 	 Saving The Model
Epoch [127/200], Loss: 0.1588
Epoch 127 		 Training Loss: 0.17807865887880325 		 Validation Loss: 1

Epoch [152/200], Loss: 0.1314
Epoch 152 		 Training Loss: 0.06572289019823074 		 Validation Loss: 1.4970956593751907
Validation Loss Decreased(3.013022--->2.994191) 	 Saving The Model
Epoch [152/200], Loss: 0.1422
Epoch 152 		 Training Loss: 0.1368033066391945 		 Validation Loss: 1.4949339777231216
Validation Loss Decreased(2.994191--->2.989868) 	 Saving The Model
Epoch [153/200], Loss: 0.1418
Epoch 153 		 Training Loss: 0.070924311876297 		 Validation Loss: 1.4973824545741081
Epoch [153/200], Loss: 0.1281
Epoch 153 		 Training Loss: 0.13499817997217178 		 Validation Loss: 1.496594823896885
Epoch [154/200], Loss: 0.1129
Epoch 154 		 Training Loss: 0.05643872544169426 		 Validation Loss: 1.4974749237298965
Epoch [154/200], Loss: 0.1556
Epoch 154 		 Training Loss: 0.13422850891947746 		 Validation Loss: 1.4970116391777992
Epoch [155/200], Loss: 0.1661
Epoch 155 		 Training Loss: 0.08306153863668442 		 Validation Loss: 1.4885024428367615
Validation Loss Decreased(2.989868--->2.977005) 	 S

Epoch [181/200], Loss: 0.1014
Epoch 181 		 Training Loss: 0.0999969094991684 		 Validation Loss: 1.370096705853939
Epoch [182/200], Loss: 0.0926
Epoch 182 		 Training Loss: 0.046288661658763885 		 Validation Loss: 1.36179169267416
Epoch [182/200], Loss: 0.1049
Epoch 182 		 Training Loss: 0.09872616082429886 		 Validation Loss: 1.3525335118174553
Epoch [183/200], Loss: 0.0908
Epoch 183 		 Training Loss: 0.04542055353522301 		 Validation Loss: 1.3377864807844162
Validation Loss Decreased(2.696470--->2.675573) 	 Saving The Model
Epoch [183/200], Loss: 0.1048
Epoch 183 		 Training Loss: 0.09780985116958618 		 Validation Loss: 1.3312305584549904
Validation Loss Decreased(2.675573--->2.662461) 	 Saving The Model
Epoch [184/200], Loss: 0.0992
Epoch 184 		 Training Loss: 0.049608856439590454 		 Validation Loss: 1.3250064924359322
Validation Loss Decreased(2.662461--->2.650013) 	 Saving The Model
Epoch [184/200], Loss: 0.0934
Epoch 184 		 Training Loss: 0.0963035561144352 		 Validation Loss: 1.

In [12]:
with torch.no_grad():
    correct = 0
    total = 0
    for data, labels in valid_loader:
        data = data.to(device)
        labels = labels.to(device)
        outputs = net(data)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    print('Accuracy of the network on the data: {} %'.format( 100 * correct / total))



Accuracy of the network on the data: 98.11320754716981 %
